## LIBRERIAS y CONSTANTES

In [1]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv

#Implement training process
#from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat, TreeFormat_all

from functions.aux import MetricTotalAccuraccy
# Load tree transformation
from functions.transform_inference import TransformerInference

from net import NeoNet

class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)


IS CUDA AVAILABLE: True


131072

### Basic parameters

In [2]:
URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_7.2_subqueries'
model_name = 'Model_Adam'
model_path = "./" + model_name + '.pt'
x = [True,False]
active_new_data = x[0]
symbol = "ᶲ"
optimizer = "Adam"
#Este parametro sirve para elegir cierta cantidad de data ordenado por rangos de tiempo obtenidos.
## Entre más bajo menos data se seleccionara. Si es muy alto se tendran demasiados valores outliners, 
## pero si es muy bajo podría tenerse una data no representativa y se aumenta el riesgo de overfitting.
## Por otro lado min_data, simplemente da el valor minimo de tiempo de ejecución que tiene una consulta tomada
## en cuenta para hacer el modelo
#percent_of_data_or = 0.93
#min_time_or = 15
#max_time_or = 80
#percent_of_data = 1
#min_time = 5
#max_time = 80


In [3]:
epsilon = 0.75 ## EPSILON
lr_rl = 0.09 ## LEARNING RATE RL
disc_fac = 0.999 ## DISCOUNT FACTOR RL 
epoch_rl = 200 ### TOTAL EPOCH POR DATA RL
n = "_" + model_name + "_alpha_" + str(lr_rl) + "_"##SALT Nombre archivos

### Features

In [4]:
# Columns to use.
list_columns = ['total_bgps', 'triples', 'treesize', 'join', 'left_join']

In [5]:
ds_train = pd.read_csv(URL + csv_name + '_ds_train_ft.csv', engine='python', encoding='utf-8')
ds_val = pd.read_csv(URL + csv_name + '_ds_val_ft.csv', engine='python', encoding='utf-8')
ds_test = pd.read_csv(URL + csv_name + '_ds_test_ft.csv', engine='python', encoding='utf-8')
ds_rl = pd.read_csv(URL + csv_name + '_ds_rl_ft.csv', engine='python', encoding='utf-8')

In [6]:
print("ds_train.shape",ds_train.shape)
print("ds_val.shape",ds_val.shape)
print("ds_test.shape",ds_test.shape)
print("ds_rl.shape",ds_rl.shape)

ds_train.shape (7938, 15)
ds_val.shape (1641, 15)
ds_test.shape (1072, 15)
ds_rl.shape (2000, 67)


# FUNCTIONS

In [7]:
def getpredictions_info(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getpredictions_info_nojc(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}



def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def pred2index_dict(x, pred_to_index, maxcardinality):
    """
    get histogram from cardinality features. the values is normalized using the max cardinality of predicate in dataset.
    :param x: Tree data from x row sample.
    :param pred_to_index: dict with predicates and their index.
    :param maxcardinality: Max cardiniality in the dataset.
    :return: dictionary with feature json_cardinality.
    """
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query, x_test_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))

    
    
    return x_train_query, x_val_query, x_test_query

def prepare_query_level_only_json_cardinality(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    
    x_train_query = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query


def prepare_query_level_data_full(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])
    xqrl = x_rl_query.drop(columns=['json_cardinality'])
    
    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)
    x_rl_scaled = scalerx.transform(xqrl)
    
    
    x_train_query = pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)
    x_rl_query = pd.concat([pd.DataFrame(x_rl_scaled,   index=xqrl.index, columns=xqrl.columns),x_rl_query[['json_cardinality']]], axis=1)
    
    
    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query['json_cardinality'] = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query

def prepare_query_level_data_full_no_jc(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    scalerx = StandardScaler()
    columns_train, index_train = x_train_query.columns, x_train_query.index
    columns_val, index_val = x_val_query.columns, x_val_query.index
    columns_test, index_test = x_test_query.columns, x_test_query.index
    columns_rl, index_rl = x_rl_query.columns, x_rl_query.index
    
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)
    x_test_scaled = scalerx.transform(x_test_query)
    x_rl_scaled = scalerx.transform(x_rl_query)
    
    x_train_query = pd.DataFrame(x_train_scaled,   index=index_train, columns=columns_test)
    x_val_query = pd.DataFrame(x_val_scaled,   index=index_val, columns=columns_val)
    x_test_query = pd.DataFrame(x_test_scaled,   index=index_test, columns=columns_test)
    x_rl_query = pd.DataFrame(x_rl_scaled,   index=index_rl, columns=columns_rl)
    return x_train_query, x_val_query, x_test_query, x_rl_query



### TreeConv Neural Net

In [8]:
folds_execution = {}
print("Size Train: {}, Val {}".format(ds_train.shape[0], ds_val.shape[0]))

Size Train: 7938, Val 1641


In [9]:
# get query level data
x_train_query = ds_train[list_columns]
x_val_query   = ds_val[list_columns]

x_train_query_str = ds_train['query']
x_val_query_str = ds_train['query']

# get plan level datba
x_train_tree = ds_train['trees'].values
x_val_tree = ds_val['trees'].values

y_train = ds_train['time'].values
y_val = ds_val['time'].values

x_test_tree = ds_test['trees'].values
y_test = ds_test['time'].values
x_test_query   = ds_test[list_columns]

x_rl_tree = ds_rl['trees'].values
y_rl = ds_rl['time'].values
x_rl_query   = ds_rl[list_columns]

In [10]:
print("---------SHAPES-----------")
print("----------CLEAN-----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val  : {ds_val.shape}')
print(f'shape ds_test : {ds_test.shape}')
print(f'shape ds_rl : {ds_rl.shape}')
print("")
print("-----TRAIN AND VAL DATA-----")
print("----------x_query_data----------")
print(f'shape x_val_query  : {x_val_query.shape}')
print(f'shape x_train_query: {x_train_query.shape}')
print("----------x_plan_level_data----------")
print(f'shape x_val_tree  : {x_val_tree.shape}')
print(f'shape x_train_tree: {x_train_tree.shape}')
print("----------y_data------------")
print(f'shape y_val  : {y_val.shape}')
print(f'shape y_train: {y_train.shape}')
print("")
print("----------TEST DATA----------")
print(f'shape x_test_tree : {x_test_tree.shape}')
print(f'shape x_test_query: {x_test_query.shape}')
print(f'shape y_test      : {y_test.shape}')
print("-----------------------")
print("")
print("----------RL DATA----------")
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")


---------SHAPES-----------
----------CLEAN-----------
shape ds_train: (7938, 15)
shape ds_val  : (1641, 15)
shape ds_test : (1072, 15)
shape ds_rl : (2000, 67)

-----TRAIN AND VAL DATA-----
----------x_query_data----------
shape x_val_query  : (1641, 5)
shape x_train_query: (7938, 5)
----------x_plan_level_data----------
shape x_val_tree  : (1641,)
shape x_train_tree: (7938,)
----------y_data------------
shape y_val  : (1641,)
shape y_train: (7938,)

----------TEST DATA----------
shape x_test_tree : (1072,)
shape x_test_query: (1072, 5)
shape y_test      : (1072,)
-----------------------

----------RL DATA----------
shape x_rl_tree : (2000,)
shape x_rl_query: (2000, 5)
shape y_rl      : (2000,)
-----------------------


In [11]:
maxcardinality = 0
#maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
maxcardinality

0

In [12]:
data_inference = {}
bao_base = ""
with (open(bao_base + "preprocess_inference_info_" + model_name + ".pickle", "rb")) as openfile:
    while True:
        try:
            data_inference = pickle.load(openfile)
        except EOFError:
            break
# Load Scaler for query features.
scalerx = None
with (open(bao_base + "scalerx_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            scalerx = pickle.load(openfile)
        except EOFError:
            break

pipeline_inverse = None
with (open(bao_base + "pipeline_inverse_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            pipeline_inverse = pickle.load(openfile)
        except EOFError:
            break
with open('NeoNetValues_' + model_name +'.txt', 'r') as file:
    data = file.read().rstrip()
neo_data = ast.literal_eval(data)

IS CUDA AVAILABLE: True


In [13]:
def ActivationFuncReluOrLeaky(string):
    if string == "<class 'torch.nn.modules.activation.LeakyReLU'>":
        return nn.LeakyReLU
    elif string == "<class 'torch.nn.modules.activation.ReLU'>":
        return nn.ReLU
io_dim_model_data = int(neo_data['io_dim'])
query_input_size = int(neo_data['query_input_size'])
query_hidden_inputs = ast.literal_eval(neo_data['query_hidden_inputs'])
query_output = int(neo_data['query_output'])
tree_units = ast.literal_eval(neo_data['tree_units'])
tree_units_dense = ast.literal_eval(neo_data['tree_units_dense'])
tree_activation_tree = ActivationFuncReluOrLeaky(neo_data['activation_tree'])
tree_activation_dense = ActivationFuncReluOrLeaky(neo_data['activation_dense'])

def get_prediction_dataset_from_load(ds, list_columns):
    x_tree = ds["trees"].values
    x_query = ds[list_columns]
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    targets = list(ds["time"].values)
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}

def get_prediction_from_load(x_tree, x_query, targets):
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}


In [14]:
reg = NeoNet(
    io_dim_model_data,
    query_input_size,
    query_hidden_inputs,
    query_output=query_output,
    tree_units=tree_units,
    tree_units_dense=tree_units_dense,
    activation_tree=tree_activation_tree,
    activation_dense=tree_activation_dense,
)
device = torch.device("cuda")
# Loading model parameters.
reg_path = model_path
#reg.load_state_dict(torch.load(reg_path, map_location=torch.device('cpu')))
reg.load_state_dict(torch.load(reg_path))
reg.to(device)
transformer = TransformerInference(
        model=reg,
        tree_transform=data_inference["tree_transform"],
        pipeline_inverse=pipeline_inverse,
        scalerx=scalerx,
        maxcardinality=data_inference["maxcardinality"]
    )
print("Modelos Listos")

Activation function in treeConv layers: <class 'torch.nn.modules.activation.LeakyReLU'>
Activation function in tree dense output layers: <class 'torch.nn.modules.activation.ReLU'>
[BinaryTreeConvWithQData(
  (weights): Conv1d(1353, 512, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(512, 256, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(256, 128, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), DynamicPooling(), Linear(in_features=128, out_features=64, bias=True), ReLU(), Linear(in_features=64, out_features=32, bias=True), ReLU(), Linear(in_features=32, out_features=1, bias=True)]
Modelos Listos


In [15]:
 ds_test['time']

0         1.687920
1         1.678266
2         1.109980
3         1.858232
4         1.598426
           ...    
1067    107.000000
1068    101.000000
1069    123.000000
1070    120.000000
1071    138.003726
Name: time, Length: 1072, dtype: float64

In [16]:
results = get_prediction_from_load(ds_test['trees'].values, ds_test[list_columns], ds_test['time'])

In [17]:
results

{'pred': [12.686357498168945,
  4.89743709564209,
  14.758463859558105,
  14.758463859558105,
  17.954404830932617,
  1.5937821865081787,
  14.75410270690918,
  1.5922574996948242,
  37.32883071899414,
  14.758463859558105,
  14.758463859558105,
  53.39189147949219,
  2.8031108379364014,
  14.758463859558105,
  14.758463859558105,
  14.758463859558105,
  44.663673400878906,
  4.89743709564209,
  1.6255521774291992,
  1.4945485591888428,
  14.758463859558105,
  1.5937821865081787,
  27.82819366455078,
  1.5922574996948242,
  36.944313049316406,
  13.64535903930664,
  2.7949717044830322,
  1.6255521774291992,
  36.944313049316406,
  14.758463859558105,
  14.758054733276367,
  84.42827606201172,
  1.5922574996948242,
  14.758463859558105,
  3.980473518371582,
  14.758463859558105,
  13.64535903930664,
  10.707067489624023,
  2.7885539531707764,
  4.291519641876221,
  6.078582763671875,
  38.94996643066406,
  2.8239212036132812,
  14.758463859558105,
  24.007583618164062,
  14.758463859558

## Reinforcement Learning

In [18]:
def GetTriplesSubtree(subtree_as_str):
    code_tpf = ['VAR_VAR_VAR', 'VAR_VAR_URI', 'VAR_URI_VAR', 'VAR_URI_URI', 'VAR_URI_LITERAL', 'VAR_VAR_LITERAL',
                'URI_URI_LITERAL', 'URI_URI_VAR', 'URI_URI_URI', 'URI_VAR_VAR', 'URI_VAR_URI', 'URI_VAR_LITERAL',
                'LITERAL_URI_VAR', 'LITERAL_URI_URI', 'LITERAL_URI_LITERAL']
    total_triples = 0
    for i in code_tpf:
        total_triples += subtree_as_str.count(i)
    #if subtree_as_str in code_tpf:
    #    total_triples += 1
    return total_triples

def GetTreeSize(subtrees, treesize):
    if len(subtrees) == 1:
        return treesize
    else:
        treesize += 1
        left_treesize = GetTreeSize(subtrees[1], treesize)
        right_treesize = GetTreeSize(subtrees[2], treesize)
        treesize = max(left_treesize,right_treesize)
    return treesize

def GetAllJoins(subtree_as_str):
    join = subtree_as_str.count('JOIN')
    left_join = subtree_as_str.count('LEFT_JOIN')
    return join-left_join, left_join

def GetIter(subtree_as_str):
    if "iter" in subtree_as_str:
        return 1
    else:
        return 0
    
def GetTotalBgp(state):
    bgp_list = []
    for s in state:
        bgp_list.append(s[1])
    bgp_list = set(bgp_list)
    return len(bgp_list)

def GetDataframe(subtree, state, columns):
    subtree_str = str(subtree)
    subtree_list = ast.literal_eval(subtree[0].tolist())
    total_bgps = GetTotalBgp(state)
    treesize = GetTreeSize(subtree_list, 1)
    triples = GetTriplesSubtree(subtree_str)
    join, left_join = GetAllJoins(subtree_str)
    iters = GetIter(subtree_str)
    
    values = [total_bgps, triples, treesize, join, left_join]
    x_rl_query = pd.DataFrame([values], columns = columns)
    return x_rl_query
    #print(x_rl_query)
    #scalerx = StandardScaler()
    #values_scaled = scalerx.fit_transform(x_rl_query)
    #x_rl_query = pd.DataFrame(values_scaled, columns = columns)
    #print(x_rl_query)
    
def RL_Actions(bgp):
    actions = []
    idx = 0
    for k,v in bgp.items():
        bgp_list = v['bgp_list']
        for b in v['bgp_list']:
            actions.append((idx,k,b['P'],b['triple_type']))
            idx += 1
    return actions


def RL_Initial_Step(actions):
    initial_state = []
    random_index = random.randint(0, len(actions)-1)
    random_action = actions[random_index]
    initial_state.append(random_action)
    return initial_state

def RL_available_actions(actions, current_state):
    if not current_state:
        return actions
    available_actions = sorted(list(set(actions) - set(current_state)))
    same_bgp_actions = []
    other_bgp_actions = []
    for i in available_actions:
        if current_state[-1][1] == i[1]:
            same_bgp_actions.append(i)
        else:
            other_bgp_actions.append(i)
    if same_bgp_actions:
        available_actions = same_bgp_actions
    else:
        available_actions = other_bgp_actions
    return available_actions


def RL_Argmax(array):
    argmax_list = np.argwhere(array == np.amax(array))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    return int(random.choice(argmax_list_flatten))

def RL_Argmax_available(q_value,available_actions):
    available_idx = [i[0] for i in available_actions]
    #print("available_actions",available_actions)
    #print("available_idx",available_idx)
    q_value_available = [q_value[i] for i in available_idx]
    #print("q_values",q_value, type(q_value))
    #print("q_values_available",q_value_available, type(q_value_available))
    
    argmax_list = np.argwhere(q_value_available == np.amax(q_value_available))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    #print("argmax_list",argmax_list, type(argmax_list))
    #print("argmax_list_flatten",argmax_list_flatten, type(argmax_list_flatten))
    value = int(random.choice(argmax_list_flatten))
    #print("value",value)
    
    #print("..........................")
    return value


def RL_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RL_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
    new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
    #tree_format = TreeFormat_all(new_dicto,symbol)
    tree_format = TreeFormat(new_dicto,symbol)
    #tree_format = ast.literal_eval(tree_format)
    new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
    x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
    pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
    reward -= pred['pred'][0]
    return reward, terminal

def RL_Rebuild_Dictionary(bgp, final_state):
    new_dicto = {}
    bgp_names = list(set([i[1] for i in final_state]))
    ### Keys
    for i in bgp_names:
        new_dicto[i] = {"bgp_list" : [], "opt" : bgp[i]['opt']}
    for i in final_state:
        new_dicto[i[1]]["bgp_list"].append({'P' : i[2], 'triple_type' : i[3]})

    return new_dicto


def RL_First_Policy(actions):
    actions_length = len(actions)
    return np.zeros((actions_length,actions_length))

def RL_bgp_format(ds_rl):
    bgps_rl = ds_rl['bgps'].values
    bgps_rl = [ast.literal_eval(bgp) for bgp in bgps_rl]
    return bgps_rl

def RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx):
    x_rl_tree_c = x_rl_tree.copy()
    x_rl_query_c = x_rl_query.copy()
    y_rl_c = y_rl.copy()
    bgps_rl_c = bgps_rl.copy()
    x_rl_query_c = x_rl_query_c.reset_index(drop=True)
    columns = x_rl_query_c.columns
    values = x_rl_query_c.values[idx]    
    x_rl_tree_test = np.array([x_rl_tree_c[idx]])
    x_rl_query_test = pd.DataFrame([values],columns=columns)
    y_rl_test = np.array([y_rl_c[idx]])
    bgps_test = bgps_rl_c[idx]
    return x_rl_tree_test, x_rl_query_test, y_rl_test, bgps_test


def RL_get_final_state_bgp_tree(q_value,actions,bgp,symbol):
    q_value = q_value.tolist()
    best_state = []
    arg_max = RL_Argmax(q_value[0])
    best_state.append(actions[arg_max])
        
    for q_val in q_value[1:]:
        available_actions = RL_available_actions(actions, best_state)
        chosen_action_available_idx = RL_Argmax_available(q_val,available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        best_state.append(chosen_action) 
    new_dicto = RL_Rebuild_Dictionary(bgp, best_state)
    #new_tree = np.array([str(TreeFormat_all(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    new_tree = np.array([str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    
    return best_state, new_dicto, new_tree


def RL_results_functions(idx,pred_old, pred_new,prl_tol1,prl_tol2):
    difference = pred_new - pred_old
    abs_diff = np.abs(difference)
    best_of_the_pred = []
    similar_of_the_pred = []
    worst_of_the_pred = []
    if abs_diff < prl_tol1:
        best_of_the_pred.append(idx)
    elif abs_diff < prl_tol2:
        similar_of_the_pred.append(idx)
    else:
        worst_of_the_pred.append(idx)
    return best_of_the_pred, similar_of_the_pred, worst_of_the_pred
    
def RLNeo_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RLNeo_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    reward = 0
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
        new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
        tree_format = TreeFormat(new_dicto,symbol)
        #tree_format = ast.literal_eval(tree_format)
        new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
        x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
        pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
        reward -= pred['pred'][0]
    return reward, terminal
    
    
def RLNeo_q_values(q_value_neo,current_state, reward):
    for q_val, act in zip(q_value_neo,current_state):
        if q_val[act[0]] == 0:
            q_val[act[0]] = reward
        else:
            q_val[act[0]] = max(reward,q_val[act[0]])
    return q_value_neo

def RL_max_available_actions(q_value, old_states, actions):
    available_actions = RL_available_actions(actions, old_states)
    index_av = [i[0] for i in available_actions]
    q_value_acotado = q_value[index_av[0]:index_av[-1]+1]
    return np.max(q_value_acotado)

In [19]:
print("----------RL DATA----------")
print(f'shape ds_rl : {ds_rl.shape}')
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")
bgps_rl = RL_bgp_format(ds_rl)

----------RL DATA----------
shape ds_rl : (2000, 67)
shape x_rl_tree : (2000,)
shape x_rl_query: (2000, 5)
shape y_rl      : (2000,)
-----------------------


## MULTI-VAL

##### Seleccionar data

In [20]:
min_num_actions = -1
buenos_idx = []
aaa = []
for i in range(0,x_rl_tree.shape[0]):
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,i)
    actions = RL_Actions(bgp_ind)
    if len(actions) >= min_num_actions:
        aaa.append(y_rl_ind)
        buenos_idx.append(i)
total_buenos_idx = len(buenos_idx)
print("total_idx",total_buenos_idx)
rl_columns = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree',
            #'history_old',
            #'history_new'
             ]
 
iteration = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree'
             ]

total_idx 2000


##### Parameters

In [21]:
#epsilon = 0.7 ## EPSILON
#lr_rl = 0.9 ## LEARNING RATE
#disc_fac = 0.999 ## DISCOUNT FACTOR
#epoch_rl = 200 ### TOTAL EPOCH POR DATA
#n = model_name ##SALT Nombre archivos

### Q - Learning - multival

##### Parameters

In [22]:
epsilon_ql = epsilon ## EPSILON
lr_ql = lr_rl ## LEARNING RATE
disc_fac_ql = disc_fac ## DISCOUNT FACTOR
epoch_ql = epoch_rl ### TOTAL EPOCH POR DATA

##### Loop

In [23]:
values_ql = []
values_final_ql = []
c_ql = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_ql = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_ql % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_ql}/{total_buenos_idx}, index {idx} evaluating')
    ####################################Q LEARNING###############################################
    for i in range(0,epoch_ql):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,   
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_ql,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_ql)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_ql,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_ql)
            
            new_state = len(current_state)-1
            old_q_value = q_value_ql[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_ql[new_state,:], old_states_val, actions)
            temporal_difference = reward + disc_fac_ql * q_max - old_q_value
            #temporal_difference = reward + disc_fac_ql * np.max(q_value_ql[new_state,:]) - old_q_value
            new_q_value = old_q_value + lr_ql * temporal_difference
            q_value_ql[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_ql[old_state,prev_action]
        new_q_value = old_q_value + lr_ql * (reward - old_q_value)
        q_value_ql[old_state,prev_action] = new_q_value
        ################################################################################################
        
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_ql.append(row)
    values_final_ql.append(row)
    c_ql += 1
print("len(values_ql[0])", len(values_ql[0]))
print("len(values_ql)", len(values_ql))

Start: 2022-01-03 18:06:55.494458
2022-01-03 18:07:41.153868 : 10/2000, index 9 evaluating
2022-01-03 18:08:33.830447 : 20/2000, index 19 evaluating
2022-01-03 18:09:28.003405 : 30/2000, index 29 evaluating
2022-01-03 18:10:19.840463 : 40/2000, index 39 evaluating
2022-01-03 18:11:13.460020 : 50/2000, index 49 evaluating
2022-01-03 18:12:10.582695 : 60/2000, index 59 evaluating
2022-01-03 18:12:54.232994 : 70/2000, index 69 evaluating
2022-01-03 18:13:53.200256 : 80/2000, index 79 evaluating
2022-01-03 18:14:41.154766 : 90/2000, index 89 evaluating
2022-01-03 18:15:33.026440 : 100/2000, index 99 evaluating
2022-01-03 18:16:35.698132 : 110/2000, index 109 evaluating
2022-01-03 18:17:37.348581 : 120/2000, index 119 evaluating
2022-01-03 18:18:31.719235 : 130/2000, index 129 evaluating
2022-01-03 18:19:21.068895 : 140/2000, index 139 evaluating
2022-01-03 18:20:14.819383 : 150/2000, index 149 evaluating
2022-01-03 18:21:03.965047 : 160/2000, index 159 evaluating
2022-01-03 18:22:03.530051

2022-01-03 20:09:49.055630 : 1370/2000, index 1369 evaluating
2022-01-03 20:10:47.116415 : 1380/2000, index 1379 evaluating
2022-01-03 20:11:43.013864 : 1390/2000, index 1389 evaluating
2022-01-03 20:12:50.539416 : 1400/2000, index 1399 evaluating
2022-01-03 20:13:52.104645 : 1410/2000, index 1409 evaluating
2022-01-03 20:14:42.267906 : 1420/2000, index 1419 evaluating
2022-01-03 20:15:36.893708 : 1430/2000, index 1429 evaluating
2022-01-03 20:16:34.398508 : 1440/2000, index 1439 evaluating
2022-01-03 20:17:28.810597 : 1450/2000, index 1449 evaluating
2022-01-03 20:18:48.679039 : 1460/2000, index 1459 evaluating
2022-01-03 20:19:46.034794 : 1470/2000, index 1469 evaluating
2022-01-03 20:20:46.518517 : 1480/2000, index 1479 evaluating
2022-01-03 20:21:44.150551 : 1490/2000, index 1489 evaluating
2022-01-03 20:22:40.348307 : 1500/2000, index 1499 evaluating
2022-01-03 20:23:35.144985 : 1510/2000, index 1509 evaluating
2022-01-03 20:24:29.459515 : 1520/2000, index 1519 evaluating
2022-01-

In [24]:
ql_df = pd.DataFrame(values_ql,columns=rl_columns)
ql_df_iteration = ql_df[iteration]
ql_df_iteration.to_csv(URL + 'rl_csvs/ql_df_iteration'+str(n)+'.csv',index=False)

In [25]:
ql_df_final = pd.DataFrame(values_final_ql,columns=rl_columns)
ql_df_final.to_csv(URL + 'rl_csvs/ql_df_final'+str(n)+'.csv',index=False)

In [26]:
ql_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.178604,79.877853,49.0,49.0,51.532357,953.441830,7.178604,30.877853,7.178604,30.877853,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.174461,78.197250,49.0,49.0,51.472896,852.479429,7.174461,29.197250,7.174461,29.197250,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
2,0,2,56.180347,78.592529,49.0,49.0,51.557389,875.717790,7.180347,29.592529,7.180347,29.592529,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
3,0,3,56.264114,56.191692,49.0,49.0,52.767358,51.720439,7.264114,7.191692,7.264114,7.191692,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.249290,56.176804,49.0,49.0,52.552212,51.506510,7.249290,7.176804,7.249290,7.176804,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,110.712654,30.428152,124.0,124.0,176.553561,8755.690722,13.287346,93.571848,13.287346,93.571848,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,111.674110,30.745872,124.0,124.0,151.927554,8696.332296,12.325890,93.254128,12.325890,93.254128,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,110.182610,30.788437,124.0,124.0,190.920279,8688.395497,13.817390,93.211563,13.817390,93.211563,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,110.205521,30.773026,124.0,124.0,190.287661,8691.268772,13.794479,93.226974,13.794479,93.226974,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [27]:
ql_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.280910,56.183128,49.0,49.0,53.011658,51.597333,7.280910,7.183128,7.280910,7.183128,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.229088,58.371490,46.0,46.0,149.550589,153.053777,12.229088,12.371490,12.229088,12.371490,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.773506,59.826965,48.0,48.0,163.162460,139.877109,12.773506,11.826965,12.773506,11.826965,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.018570,105.704468,49.0,49.0,3846.303018,3215.396665,62.018570,56.704468,62.018570,56.704468,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,138.666458,78.189354,111.0,111.0,765.432905,1076.538495,27.666458,32.810646,27.666458,32.810646,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1996,1996,199,101.758583,105.044403,122.0,122.0,409.714959,287.492267,20.241417,16.955597,20.241417,16.955597,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""LEFT_JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct..."
1997,1997,199,133.066757,73.618034,109.0,109.0,579.208802,1251.883492,24.066757,35.381966,24.066757,35.381966,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.541382,68.907028,136.0,136.0,508.113895,4501.466865,22.541382,67.092972,22.541382,67.092972,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## Sarsa Multival

###### Parameters

In [28]:
epsilon_sar = epsilon ## EPSILON
lr_sar = lr_rl ## LEARNING RATE
disc_fac_sar = disc_fac ## DISCOUNT FACTOR
epoch_sar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [29]:
values_sar = []
values_final_sar = []
c_sar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_sar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_sar}/{total_buenos_idx}, index {idx} evaluating')
    ############################ALGORITMO SARSA##########################################
    for i in range(0,epoch_sar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_sar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_sar)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_sar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_sar)

            new_state = len(current_state)-1
            old_q_value = q_value_sar[old_state,prev_action]
            temporal_difference = reward + (disc_fac_sar * q_value_sar[new_state,chosen_action_idx]) - old_q_value
            new_q_value = old_q_value + lr_sar * temporal_difference
            q_value_sar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_sar[old_state,prev_action]
        new_q_value = old_q_value + lr_sar * (reward - old_q_value)
        q_value_sar[old_state,prev_action] = new_q_value
        ###############################################################################################33
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_sar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_sar.append(row)
    values_final_sar.append(row)
    c_sar += 1
#print("values_sar ", values_sar)
print("len(values_sar[0])",len(values_sar[0]) )
print("len(values_sar)", len(values_sar))

Start: 2022-01-03 21:12:58.721573
2022-01-03 21:13:48.279668 : 10/2000, index 9 evaluating
2022-01-03 21:14:44.379834 : 20/2000, index 19 evaluating
2022-01-03 21:15:40.161449 : 30/2000, index 29 evaluating
2022-01-03 21:16:32.787613 : 40/2000, index 39 evaluating
2022-01-03 21:17:26.500217 : 50/2000, index 49 evaluating
2022-01-03 21:18:25.286465 : 60/2000, index 59 evaluating
2022-01-03 21:19:09.388895 : 70/2000, index 69 evaluating
2022-01-03 21:20:09.435808 : 80/2000, index 79 evaluating
2022-01-03 21:20:58.367879 : 90/2000, index 89 evaluating
2022-01-03 21:21:51.350248 : 100/2000, index 99 evaluating
2022-01-03 21:22:54.861923 : 110/2000, index 109 evaluating
2022-01-03 21:23:57.959181 : 120/2000, index 119 evaluating
2022-01-03 21:24:52.952206 : 130/2000, index 129 evaluating
2022-01-03 21:26:37.986967 : 150/2000, index 149 evaluating
2022-01-03 21:27:28.552463 : 160/2000, index 159 evaluating
2022-01-03 21:28:29.304128 : 170/2000, index 169 evaluating
2022-01-03 21:29:22.716362

2022-01-03 23:20:49.074101 : 1400/2000, index 1399 evaluating
2022-01-03 23:21:51.221929 : 1410/2000, index 1409 evaluating
2022-01-03 23:22:41.768831 : 1420/2000, index 1419 evaluating
2022-01-03 23:23:36.684919 : 1430/2000, index 1429 evaluating
2022-01-03 23:24:34.652583 : 1440/2000, index 1439 evaluating
2022-01-03 23:25:29.471431 : 1450/2000, index 1449 evaluating
2022-01-03 23:26:49.744796 : 1460/2000, index 1459 evaluating
2022-01-03 23:27:47.524821 : 1470/2000, index 1469 evaluating
2022-01-03 23:28:48.366446 : 1480/2000, index 1479 evaluating
2022-01-03 23:29:45.992681 : 1490/2000, index 1489 evaluating
2022-01-03 23:30:42.350762 : 1500/2000, index 1499 evaluating
2022-01-03 23:31:37.402305 : 1510/2000, index 1509 evaluating
2022-01-03 23:32:32.279209 : 1520/2000, index 1519 evaluating
2022-01-03 23:33:30.566171 : 1530/2000, index 1529 evaluating
2022-01-03 23:34:22.783522 : 1540/2000, index 1539 evaluating
2022-01-03 23:35:20.501977 : 1550/2000, index 1549 evaluating
2022-01-

In [30]:
sarsa_df = pd.DataFrame(values_sar,columns=rl_columns)
sarsa_df_iteration = sarsa_df[iteration]
sarsa_df_iteration.to_csv(URL + 'rl_csvs/sarsa_df_iteration'+str(n)+'.csv',index=False)

In [31]:
sarsa_df_final = pd.DataFrame(values_final_sar,columns=rl_columns)
sarsa_df_final.to_csv(URL + 'rl_csvs/sarsa_df_final'+str(n)+'.csv',index=False)

In [32]:
sarsa_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.263817,56.195621,49.0,49.0,52.763035,51.776969,7.263817,7.195621,7.263817,7.195621,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,0,1,55.945801,56.313396,49.0,49.0,48.244148,53.485768,6.945801,7.313396,6.945801,7.313396,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
2,0,2,56.194611,56.193737,49.0,49.0,51.762422,51.749852,7.194611,7.193737,7.194611,7.193737,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
3,0,3,56.178524,56.178059,49.0,49.0,51.531207,51.524526,7.178524,7.178059,7.178524,7.178059,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.230377,56.347622,49.0,49.0,52.278354,53.987548,7.230377,7.347622,7.230377,7.347622,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,110.240311,30.927656,124.0,124.0,189.329050,8662.461185,13.759689,93.072344,13.759689,93.072344,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,111.632965,30.916134,124.0,124.0,152.943553,8664.606132,12.367035,93.083866,12.367035,93.083866,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,109.185272,30.835651,124.0,124.0,219.476159,8679.595850,14.814728,93.164349,14.814728,93.164349,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,107.488182,30.874249,124.0,124.0,272.640131,8672.405592,16.511818,93.125751,16.511818,93.125751,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [33]:
sarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.756798,56.174187,49.0,49.0,60.167912,51.468955,7.756798,7.174187,7.756798,7.174187,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,59.466522,58.834236,46.0,46.0,181.347221,164.717617,13.466522,12.834236,13.466522,12.834236,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,59.922695,60.498005,48.0,48.0,142.150660,156.200127,11.922695,12.498005,11.922695,12.498005,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,46.054855,48.0,48.0,5.897155,3.783588,2.428406,1.945145,2.428406,1.945145,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.021027,111.021614,49.0,49.0,3846.607742,3846.680612,62.021027,62.021614,62.021027,62.021614,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.191772,137.705841,111.0,111.0,794.776034,713.201947,28.191772,26.705841,28.191772,26.705841,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,102.544434,101.542336,122.0,122.0,378.519064,418.516036,19.455566,20.457664,19.455566,20.457664,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""LEFT_JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct..."
1997,1997,199,133.130447,75.398735,109.0,109.0,582.278491,1129.045006,24.130447,33.601265,24.130447,33.601265,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.484177,71.536377,136.0,136.0,505.538199,4155.558696,22.484177,64.463623,22.484177,64.463623,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## Expected-Sarsa Multival

###### Parameters

In [34]:
epsilon_exsar = epsilon ## EPSILON
epsilon_exsar_alg = epsilon
lr_exsar = lr_rl ## LEARNING RATE
disc_fac_exsar = disc_fac ## DISCOUNT FACTOR
epoch_exsar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [35]:
values_exsar = []
values_final_exsar = []
c_exsar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_exsar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_exsar}/{total_buenos_idx}, index {idx} evaluating')
        
    ##### ALGORITMO EXPECTED SARSA
    for i in range(0,epoch_exsar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_exsar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_exsar)    
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_exsar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_exsar)
            new_state = len(current_state)-1
            array_state = np.array(current_state)




            expected_q = 0
            old_q_value = q_value_exsar[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_exsar[new_state,:], old_states_val, actions)
            #q_max = np.max(q_value_exsar[new_state,:])
            pi = np.ones(num_actions)*epsilon_exsar_alg/num_actions \
                + (q_value_exsar[new_state,:] == q_max)*(1-epsilon_exsar_alg)/np.sum(q_value_exsar[new_state,:] == q_max)
    
            expected_q = np.sum(q_value_exsar[new_state,:]*pi)
            temporal_difference = reward + disc_fac_exsar * expected_q - old_q_value
            new_q_value = old_q_value +  lr_exsar*temporal_difference
            q_value_exsar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        #########################################################
        old_q_value = q_value_exsar[old_state,prev_action]
        new_q_value = old_q_value + lr_exsar * (reward - old_q_value)
        q_value_exsar[old_state,prev_action] = new_q_value

        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_exsar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_exsar.append(row)
    values_final_exsar.append(row)
    c_exsar += 1
#print("values_exsar ", values_exsar)
print("len(values_exsar[0])",len(values_exsar[0]) )
print("len(values_exsar)", len(values_exsar))

Start: 2022-01-04 00:21:21.801004
2022-01-04 00:22:11.642053 : 10/2000, index 9 evaluating
2022-01-04 00:23:07.921618 : 20/2000, index 19 evaluating
2022-01-04 00:24:03.500869 : 30/2000, index 29 evaluating
2022-01-04 00:24:57.106385 : 40/2000, index 39 evaluating
2022-01-04 00:25:51.899359 : 50/2000, index 49 evaluating
2022-01-04 00:26:50.688342 : 60/2000, index 59 evaluating
2022-01-04 00:27:35.371035 : 70/2000, index 69 evaluating
2022-01-04 00:28:36.151670 : 80/2000, index 79 evaluating
2022-01-04 00:29:25.565670 : 90/2000, index 89 evaluating
2022-01-04 00:30:18.306036 : 100/2000, index 99 evaluating
2022-01-04 00:31:22.468139 : 110/2000, index 109 evaluating
2022-01-04 00:32:26.139166 : 120/2000, index 119 evaluating
2022-01-04 00:33:21.215293 : 130/2000, index 129 evaluating
2022-01-04 00:34:11.889771 : 140/2000, index 139 evaluating
2022-01-04 00:35:07.196971 : 150/2000, index 149 evaluating
2022-01-04 00:35:57.779832 : 160/2000, index 159 evaluating
2022-01-04 00:36:58.921625

2022-01-04 02:27:14.791394 : 1370/2000, index 1369 evaluating
2022-01-04 02:28:12.992437 : 1380/2000, index 1379 evaluating
2022-01-04 02:29:09.608260 : 1390/2000, index 1389 evaluating
2022-01-04 02:30:18.108697 : 1400/2000, index 1399 evaluating
2022-01-04 02:31:21.009885 : 1410/2000, index 1409 evaluating
2022-01-04 02:32:12.100888 : 1420/2000, index 1419 evaluating
2022-01-04 02:33:07.241618 : 1430/2000, index 1429 evaluating
2022-01-04 02:34:05.625200 : 1440/2000, index 1439 evaluating
2022-01-04 02:35:00.787619 : 1450/2000, index 1449 evaluating
2022-01-04 02:36:22.531147 : 1460/2000, index 1459 evaluating
2022-01-04 02:37:20.845916 : 1470/2000, index 1469 evaluating
2022-01-04 02:38:21.942277 : 1480/2000, index 1479 evaluating
2022-01-04 02:39:19.638114 : 1490/2000, index 1489 evaluating
2022-01-04 02:40:16.548959 : 1500/2000, index 1499 evaluating
2022-01-04 02:41:12.074716 : 1510/2000, index 1509 evaluating
2022-01-04 02:42:07.068084 : 1520/2000, index 1519 evaluating
2022-01-

In [36]:
exsarsa_df = pd.DataFrame(values_exsar,columns=rl_columns)
exsarsa_df_iteration = exsarsa_df[iteration]
exsarsa_df_iteration.to_csv(URL + 'rl_csvs/exsarsa_df_iteration'+str(n)+'.csv',index=False)

In [37]:
exsarsa_df_final = pd.DataFrame(values_final_exsar,columns=rl_columns)
exsarsa_df_final.to_csv(URL + 'rl_csvs/exsarsa_df_final'+str(n)+'.csv',index=False)

In [38]:
exsarsa_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.195866,79.334145,49.0,49.0,51.780482,920.160328,7.195866,30.334145,7.195866,30.334145,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.183922,77.389389,49.0,49.0,51.608733,805.957410,7.183922,28.389389,7.183922,28.389389,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
2,0,2,56.243149,83.209328,49.0,49.0,52.463205,1170.278102,7.243149,34.209328,7.243149,34.209328,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
3,0,3,56.229939,55.973206,49.0,49.0,52.272011,48.625596,7.229939,6.973206,7.229939,6.973206,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.177021,56.175331,49.0,49.0,51.509631,51.485377,7.177021,7.175331,7.177021,7.175331,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.667503,30.923851,124.0,124.0,152.090473,8663.169510,12.332497,93.076149,12.332497,93.076149,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,109.607025,30.973934,124.0,124.0,207.157725,8653.848923,14.392975,93.026066,14.392975,93.026066,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,110.402740,30.500116,124.0,124.0,184.885466,8742.228243,13.597260,93.499884,13.597260,93.499884,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,111.542938,30.582054,124.0,124.0,155.178388,8726.912609,12.457062,93.417946,12.457062,93.417946,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [39]:
exsarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.220879,56.194256,49.0,49.0,52.141088,51.757317,7.220879,7.194256,7.220879,7.194256,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,57.486568,58.666508,46.0,46.0,131.941255,160.440418,11.486568,12.666508,11.486568,12.666508,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.917332,60.620293,48.0,48.0,166.857458,159.271787,12.917332,12.620293,12.917332,12.620293,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.034111,111.012428,49.0,49.0,3848.230930,3845.541262,62.034111,62.012428,62.034111,62.012428,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.728653,139.634674,111.0,111.0,825.335501,819.944559,28.728653,28.634674,28.728653,28.634674,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,103.062172,101.085831,122.0,122.0,358.641332,437.402478,18.937828,20.914169,18.937828,20.914169,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""LEFT_JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct..."
1997,1997,199,133.099426,73.803215,109.0,109.0,580.782347,1238.813672,24.099426,35.196785,24.099426,35.196785,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.696030,71.724846,136.0,136.0,515.109762,4131.295436,22.696030,64.275154,22.696030,64.275154,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


In [40]:
1+1

2

### Archivos enteros

In [41]:
import os
path = "/home/school/math/final_analysis.csv"

In [42]:
os.remove(URL + 'rl_csvs/ql_df.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/media/data/ccarmona/memoria/dataset/rl_csvs/ql_df.csv'

In [ ]:
os.remove(URL + 'rl_csvs/sarsa_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/neo_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/exsarsa_df.csv')

In [ ]:
ql_df.to_csv(URL + 'rl_csvs/ql_df.csv',index=False)

In [ ]:
sarsa_df.to_csv(URL + 'rl_csvs/sarsa_df.csv',index=False)

In [ ]:
neo_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)

In [ ]:
exsarsa_df.to_csv(URL + 'rl_csvs/exsarsa_df.csv',index=False)